<a href="https://colab.research.google.com/github/Sanjanathakal/MA630/blob/master/Assignment_7_Question_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sympy import *
init_printing(use_unicode=True)
import json
import matplotlib.pyplot as plt
import pandas as pd
import sys
from typing import Callable, List, Tuple, Union

In [ ]:
x = IndexedBase("x")
i = symbols("i")
C = symbols("C")

In [ ]:
def func():
    A =x[1] - 3 
    B = x[2] - 5
    D = x[1]**2 + x[2]**2 -10
    E = -2*x[1] + x[2] -5
    f = - A**2 - B**2 + ((C/2) * (( D**2 ) + ( E**2 )))
    return f

In [ ]:
fn=func()
print(fn)

C*((-2*x[1] + x[2] - 5)**2 + (x[1]**2 + x[2]**2 - 10)**2)/2 - (x[1] - 3)**2 - (x[2] - 5)**2


In [ ]:
def diff(element, function):
    partial_diff = function.diff(element)
    return partial_diff

In [ ]:
def gradient(n,function):  
    partials = []
    for j in range(1,3):
        partial_diff = diff(x[j], function)
        partials.append(partial_diff)
    return partials

In [ ]:
g = gradient(2,fn)

In [ ]:
print(g)

[C*(4*(x[1]**2 + x[2]**2 - 10)*x[1] + 8*x[1] - 4*x[2] + 20)/2 - 2*x[1] + 6, C*(4*(x[1]**2 + x[2]**2 - 10)*x[2] - 4*x[1] + 2*x[2] - 10)/2 - 2*x[2] + 10]


In [ ]:
def direction_minimization(func: Callable, x0: List[float],d: np.ndarray,
                           sL: float, val0: float, t0: float,
                           toldir: float, itmax: int = 50, cut: float = 0.1,
                           increase: int = 3) -> Tuple[float, float]:

    # itmax = 50
    # cut = 0.1
    # increase = 3
    small = 1E-8 * t0

    tR = 0
    tL = 0
    # sL = s0
    vL = val0
    vR = val0
    val = val0

    t = t0
    if t0 < 0:
        t = 1

    for i in range(itmax):

        x = x0 + t*d
        val, grad = banana(x)
        s = np.dot(d, grad)
        if (abs(s) < toldir) and (val < val0):
            break
        if (s > 0) or (val > val0):
            tR = t
            sR = s
            vR = val
        else:
            tL = t
            sL = s
            vL = val

        if tR > 0:
            a = sL + sR - 3*(vR - vL)/(tR - tL)
            b = np.sqrt(a**2 - sL*sR)
            t = tR - (tR - tL)*(sR + b - a)/(sR - sL + 2*b)
            t = max(t, tL + cut*(tR - tL))
            t = min(t, tL + (1 - cut)*(tR - tL))
        else:
             t = increase*t

        if t < small:
            break

    if vL < val:
        t = tL
        val = vL
        x = x0 + tL*d

    if vR < val:
        t = tR
        val = vR
        x = x0 + tR*d

    if val >= val0:
        t = 0
        x = x0

    return x, t

In [ ]:
def conjugate_gradient(func: Callable, x: List[float],step_size: float,
                       max_iter: int = 100, tolerance: float = 1E-4
                       ) -> List[np.ndarray]:
    x = np.array(x)
    point_trajectory = [x]
    cur_step_size = step_size
    step_size_tol = 1.0E-10

    max_cycles = min(10, x.size)
    cycle = 0

    prev_fun_grad = np.zeros(x.size)
    prev_direction = 0
    prev_norm_grad = 1
    for i in range(max_iter):
        fun_val, fun_grad = banana(x)
        norm_grad = np.linalg.norm(fun_grad)

        print(f'it={str(i).zfill(3)}  f={fun_val:.5f} '
              f' |grad f|={norm_grad:.5f}')

        x_values = json.dumps(np.around(x, decimals=4).tolist())
        print(f'  x = {x_values}')

        if norm_grad < tolerance:
            break

        s = -norm_grad**2
        direction = -fun_grad
        if cycle > 0:
            grad_delta = fun_grad - prev_fun_grad
            alpha = np.dot(grad_delta, fun_grad) / prev_norm_grad**2
            direction = (alpha * prev_direction) - fun_grad
            s = np.dot(direction, fun_grad)

            if s >= -tolerance**2:
                direction = -fun_grad
                s = -norm_grad**2
                cycle = 0

        prev_fun_grad = fun_grad
        prev_direction = direction
        prev_norm_grad = norm_grad

        if cur_step_size <= step_size_tol:
            cur_step_size = step_size

        norm_direction = np.linalg.norm(direction)
        toldir = tolerance * norm_direction
        init_step = cur_step_size / norm_direction
        x, cur_step_size = direction_minimization(
            func=func,
            x0=x,
            d=direction,
            sL=s,
            val0=fun_val,
            t0=init_step,
            toldir=toldir
        )

        point_trajectory += [x]

        if cycle == 0 and cur_step_size == 0:
            break
        if cycle == max_cycles or cur_step_size == 0:
            cycle = 0
        else:
            cycle += 1

        cur_step_size = cur_step_size * norm_direction

    return point_trajectory


In [ ]:
fun = func()

In [ ]:
#Sample of the equation
fn_expr = fun.doit()
print(fn_expr)

C*((-2*x[1] + x[2] - 5)**2 + (x[1]**2 + x[2]**2 - 10)**2)/2 - (x[1] - 3)**2 - (x[2] - 5)**2


In [ ]:
grad = gradient(2,fn_expr)
pprint(grad)

⎡  ⎛  ⎛    2       2     ⎞                            ⎞                 ⎛  ⎛  
⎢C⋅⎝4⋅⎝x[1]  + x[2]  - 10⎠⋅x[1] + 8⋅x[1] - 4⋅x[2] + 20⎠               C⋅⎝4⋅⎝x[
⎢────────────────────────────────────────────────────── - 2⋅x[1] + 6, ────────
⎣                          2                                                  

  2       2     ⎞                            ⎞              ⎤
1]  + x[2]  - 10⎠⋅x[2] - 4⋅x[1] + 2⋅x[2] - 10⎠              ⎥
────────────────────────────────────────────── - 2⋅x[2] + 10⎥
                  2                                         ⎦


In [ ]:
#coverting to a list:
for i in range(0,2):
    print(grad[i],",\n")

C*(4*(x[1]**2 + x[2]**2 - 10)*x[1] + 8*x[1] - 4*x[2] + 20)/2 - 2*x[1] + 6 ,

C*(4*(x[1]**2 + x[2]**2 - 10)*x[2] - 4*x[1] + 2*x[2] - 10)/2 - 2*x[2] + 10 ,



In [ ]:
#For N = 10 
def banana(x: List[float]) -> Union[float, Tuple]:
    C=100000
    fun_val = C*((-2*x[0] + x[1] - 5)**2 + (x[0]**2 + x[1]**2 - 10)**2)/2 - (x[0] - 3)**2 - (x[1] - 5)**2
    fun_grad = np.array([C*(4*(x[0]**2 + x[1]**2 - 10)*x[0] + 8*x[0] - 4*x[1] + 20)/2 - 2*x[0] + 6 ,
                         C*(4*(x[0]**2 + x[1]**2 - 10)*x[1] - 4*x[0] + 2*x[1] - 10)/2 - 2*x[1] + 10])
    
    return fun_val, fun_grad

In [ ]:
step_size = 1.0
max_iter = 100
tolerance = 1E-5

x= [3.0,5.0]
path = conjugate_gradient(banana(x),x,step_size, max_iter, tolerance)

it=000  f=30600000.00000  |grad f|=28123299.94862
  x = [3.0, 5.0]
it=001  f=1379769.36065  |grad f|=1177382.50489
  x = [1.4959, 2.7438]
it=002  f=445257.32771  |grad f|=1182606.15266
  x = [0.389, 3.4166]
it=003  f=19734.75890  |grad f|=383700.25903
  x = [-0.868, 3.1405]
it=004  f=3166.10040  |grad f|=40946.96289
  x = [-0.8493, 3.0551]
it=005  f=19.90701  |grad f|=17930.01016
  x = [-0.9974, 3.0056]
it=006  f=-19.56665  |grad f|=527.23034
  x = [-0.9982, 3.0006]
it=007  f=-19.66754  |grad f|=695.26578
  x = [-0.9986, 3.0007]
it=008  f=-20.00017  |grad f|=1.02578
  x = [-1.0, 3.0]
it=009  f=-20.00017  |grad f|=0.00018
  x = [-1.0, 3.0]
it=010  f=-20.00017  |grad f|=0.00010
  x = [-1.0, 3.0]
it=011  f=-20.00017  |grad f|=0.00010
  x = [-1.0, 3.0]
